In [ ]:
!pip -q install beautifulsoup4 lxml pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import glob
files = glob.glob("/content/drive/MyDrive/telegram_exports/*.html")
print("Found files:", files)

Found files: ['/content/drive/MyDrive/telegram_exports/messages_translated1.html']


In [ ]:
# 👉 EDIT THESE TWO PATHS
INPUT_DIR  = "/content/drive/MyDrive/telegram_exports"
OUTPUT_DIR = "/content/drive/MyDrive/html_keyword_output"  # where results will be saved

# Threshold: set 1 for "not zero", set 2 for "greater than 1"
MIN_COUNT = 1

# Optional toggles
SPLIT_PARAGRAPHS = False       # True = also write one HTML file per match
INCLUDE_ORIGINAL_HEAD = True   # Try to keep original <head> styling
LEXICON_CSV = None             # Or set to a path (e.g., "/content/drive/MyDrive/lexicon.csv")


In [ ]:
# --- Write updated extract_paragraphs.py ---
script_path = "/content/extract_paragraphs.py"

content = r'''
import os
import json
import re
from bs4 import BeautifulSoup

# ============ CONFIG ============
INPUT_DIR  = "/content/drive/MyDrive/telegram_exports"
OUTPUT_DIR = "/content/drive/MyDrive/html_keyword_output"
TARGET_FILE = "messages_translated1.html"
os.makedirs(OUTPUT_DIR, exist_ok=True)

ORIGINAL_FILE = os.path.join(INPUT_DIR, TARGET_FILE)  # fix here
original_file_url = "https://drive.google.com/file/d/1u9H-GJXpeWQH5J82vv56V4NuxFDe9mMy/view?usp=drive_link"

print("[INFO] INPUT_DIR:", INPUT_DIR)
print("[INFO] OUTPUT_DIR:", OUTPUT_DIR)
print("[INFO] TARGET_FILE:", TARGET_FILE)
print("[INFO] ORIGINAL_FILE exists?:", os.path.exists(ORIGINAL_FILE))


# ============ KEYWORDS ============
KEYWORD_VARIANTS = {
    "troop movements": ["troop", "convoy", "military movement", "forces", "caravans", "vanguard", "moving warships"],
    "airstrikes and bombing": ["airstrike", "airstrikes", "air raids", "bombing", "bomb drops", "military bombing", "drop bombs", "bombs"],
    "military aircraft": ["drone", "droppers", "aircraft", "paramotor"],
    "military engagements": ["counterattack", "revolutionary", "defeat", "destroyed", "demolished", "occupied", "attack", "blockade"],
    "weapons": ["weapon", "gun", "guns", "arms", "explosives", "missile", "pistols", "rocket launchers", "assault rifles", "AK", "RPG", "heavy artillery", "heavy weapons", "gunfire", "ammunition", "explosions"],
    "armed groups": ["rebels", "insurgents", "military", "army", "airforce", "People's Liberation Army", "Special Operations Force", "SOF", "BAMAR ARMY", "CDM"],
    "prisoners of war": ["prisoners of war", "POW"],
    "china": ["china", "chinese"],
    "organizations": ["KIA", "MNDAA", "PNO", "NUG", "NAC", "TNLA", "PDF", "UWSA", "SSPP", "NSIMS"]
}

LOCATION_TRIGGERS = ["mountain", "border", "township", "province", "region", "city", "village",
                     "valley", "river", "area", "camp", "checkpoint", "base", "zone", "station"]

def extract_paragraph_text(div_element):
    return div_element.get_text(separator="\n", strip=False)

def count_keywords(text):
    counts = {}
    lowered_text = text.lower()
    for main_keyword, variants in KEYWORD_VARIANTS.items():
        total = 0
        for variant in variants:
            pattern = re.compile(rf"(?<!\w){re.escape(variant.lower())}(?!\w)", re.IGNORECASE)
            matches = pattern.findall(lowered_text)
            total += len(matches)
        counts[main_keyword] = total
    return counts

def extract_locations(text):
    locations = []
    words = text.split()
    for i, word in enumerate(words):
        if any(loc in word for loc in LOCATION_TRIGGERS):
            snippet = " ".join(words[max(i-3,0):i+3])
            if snippet not in locations:
                locations.append(snippet)
    return locations

file_path = os.path.join(INPUT_DIR, TARGET_FILE)
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Telegram HTML not found: {file_path}")

with open(file_path, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

messages = soup.find_all("div", class_="message")
results = []

for message in messages:
    text_div = message.find("div", class_="text")
    if not text_div:
        continue
    paragraph = extract_paragraph_text(text_div)
    keyword_counts = count_keywords(paragraph)
    if not any(v>0 for v in keyword_counts.values()):
        continue
    message_id = message.get("id","N/A")
    username_div = message.find("div", class_="from_name")
    username = username_div.get_text(strip=True) if username_div else "Unknown Sender"
    time_div = message.find("div", class_="pull_right date details")
    time_text = time_div.get("title") if time_div and time_div.has_attr("title") else (time_div.get_text(strip=True) if time_div else "N/A")
    word_count = len(paragraph.split())
    score = sum(keyword_counts.values())
    results.append({
        "file": TARGET_FILE,
        "message_id": message_id,
        "username": username,
        "time": time_text,
        "text_snippet": paragraph,
        "word_count": word_count,
        "locations": extract_locations(paragraph),
        "score": round(score,6),
        "matched_keywords": keyword_counts
    })

# ---------- SORT + SAVE ----------
results_sorted = sorted(results, key=lambda x: x["score"], reverse=True)
json_out_path = os.path.join(OUTPUT_DIR, "final_paragraphs_sorted.json")
with open(json_out_path, "w", encoding="utf-8") as jf:
    json.dump(results_sorted, jf, indent=2, ensure_ascii=False)

# ---------- HTML ----------
html_parts = [
    "<!doctype html>",
    "<html><head><meta charset='utf-8'><title>Telegram Extracts</title>",
    "<style>body{font-family:Arial, sans-serif}.message{border:1px solid #ccc;padding:10px;margin:10px;border-radius:6px}.from_name{font-weight:bold;color:#2b4}.date{color:#666;font-size:0.85em}.text{margin:8px 0}.meta{font-size:0.85em;color:#666}</style>",
    "</head><body>"
]

for idx, r in enumerate(results_sorted,1):
    msg_id = r["message_id"]
    link_html = f"<a href='{original_file_url}#{msg_id}' target='_blank'>View Original</a>" if msg_id and msg_id!="N/A" else ""
    html_parts.append("<div class='message'>")
    html_parts.append(f"  <div class='from_name'>{r.get('username','Unknown Sender')}</div>")
    html_parts.append(f"  <div class='date'>{r.get('time','N/A')}</div>")
    html_parts.append(f"  <div class='text'>{r['text_snippet']}</div>")
    html_parts.append(f"  <div class='meta'>Score: {r['score']} | Words: {r.get('word_count','')} {('&nbsp;|&nbsp;' + link_html) if link_html else ''}</div>")
    html_parts.append("</div>")

html_parts.append("</body></html>")
html_out_path = os.path.join(OUTPUT_DIR, "final_paragraphs_sorted_like_telegram.html")
with open(html_out_path, "w", encoding="utf-8") as hf:
    hf.write("\n".join(html_parts))

print("✅ Extraction complete")
print("JSON saved at:", json_out_path)
print("HTML saved at:", html_out_path)
'''

with open(script_path, "w", encoding="utf-8") as f:
    f.write(content)

print("✅ Wrote", script_path)
!python /content/extract_paragraphs.py


✅ Wrote /content/extract_paragraphs.py
[INFO] INPUT_DIR: /content/drive/MyDrive/telegram_exports
[INFO] OUTPUT_DIR: /content/drive/MyDrive/html_keyword_output
[INFO] TARGET_FILE: messages_translated1.html
[INFO] ORIGINAL_FILE exists?: True
✅ Extraction complete
JSON saved at: /content/drive/MyDrive/html_keyword_output/final_paragraphs_sorted.json
HTML saved at: /content/drive/MyDrive/html_keyword_output/final_paragraphs_sorted_like_telegram.html


In [ ]:
# === Score + Sort Paragraphs (Colab-ready) ===
import os, json, re

# --- Config ---
OUTPUT_DIR = "/content/drive/MyDrive/html_keyword_output"
TARGET_FILE = "messages_translated1.html"
ORIGINAL_FILE = os.path.join("/content/drive/MyDrive/telegram_exports", TARGET_FILE)  # points to original
original_file_url = "https://drive.google.com/file/d/1u9H-GJXpeWQH5J82vv56V4NuxFDe9mMy/view?usp=drive_link"

json_file = os.path.join(OUTPUT_DIR, "final_paragraphs_sorted.json")
summarized_html_file = os.path.join(OUTPUT_DIR, "final_paragraphs_with_summary.html")

# Debug prints
print("[DEBUG] OUTPUT_DIR:", OUTPUT_DIR)
print("[DEBUG] TARGET_FILE:", TARGET_FILE)
print("[DEBUG] ORIGINAL_FILE exists?:", os.path.exists(ORIGINAL_FILE))
print("[DEBUG] JSON file path:", json_file)
print("[DEBUG] Summarized HTML file path:", summarized_html_file)

# ---------- KEYWORD VARIANTS (with weights for scoring) ----------
KEYWORD_VARIANTS = {
    "troop movements": ["troop:0.01","convoy:0.6","military movement:0.4","forces:0.01","caravans:0.6","vanguard:0.6","moving warships:0.7"],
    "airstrikes and bombing": ["airstrike:0.7","airstrikes:0.7","air raids:0.7","bombing:0.7","bomb drops:0.7","military bombing:0.7","drop bombs:0.7","bombs:0.6"],
    "military aircraft": ["drone:0.7","droppers:0.7","aircraft:0.7","paramotor:0.8"],
    "military engagements": ["counterattack:0.8","revolutionary:0.1","defeat:0.1","destroyed:0.2","demolished:0.4","occupied:0.4","blockade:0.3"],
    "weapons": ["weapon:0.5","gun:0.5","guns:0.5","arms:0.5","explosives:0.7","missile:0.7","pistols:0.5","rocket launchers:0.7","assault rifles:0.5","AK:0.6","RPG:0.7","heavy artillery:0.7","heavy weapons:0.7","gunfire:0.5","ammunition:0.5","explosions:0.5"],
    "armed groups": ["rebels:0.2","insurgents:0.2","military:0.1","army:0.1","airforce:0.5","People's Liberation Army:0.6","Special Operations Force:0.7","SOF:0.7","BAMAR ARMY:0.7","CDM:0.7"],
    "prisoners of war": ["prisoners of war:0.7","POW:0.7"],
    "china": ["china:0.2","chinese:0.2"],
    "thailand border": ["thailand border:0.4","thai border:0.5"],
    "myanmar border": ["myanmar border:0.3","burma border:0.3"],
    "china border": ["china border:0.3"],
    "india-myanmar border": ["india-myanmar border region:0.6","northeast india:0.6","manipur border:0.6"],
    "geopolitics": ["Myanmar-Thailand Armed Forces meet:0.7","special envoy on Asian affairs of the Chinese Ministry of Foreign Affairs:0.7","Myanmar-China-Thailand:0.7","Myanmar-Japan-Thailand:0.7","Ambassador:0.6"],
    "human trafficking": ["human trafficking:0.5","trafficked people:0.5"],
    "drugs": ["drugs:0.6","narcotics:0.6","meth:0.6","opium:0.6","anti-narcotics:0.6"],
    "gambling": ["gambling:0.6","casino:0.6","betting:0.6","online gambling:0.6"],
    "fraud": ["fraud:0.5","scam:0.5","scamming:0.5","telecom fraud:0.6","online fraud:0.6","internet scam:0.6","digital fraud:0.6","online scams:0.6","online financial fraud:0.7","Online Scam Center:0.7"],
    "smuggling": ["smuggling gold:0.7"],
    "forced conscription": ["forced conscription:0.7","conscripted:0.5"],
    "organizations": ["KIA:1","MNDAA:1","PNO:1","NUG:1","NAC:1","TNLA:1","PDF:1","Kapas:1","MOGE:1","CRPH:1","OWIC:1","ASEAN:1",
                      "KNU:1","NLD:1","KTV:1","BAMAR ARMY:1","CDM:1","PSMS:1","KNLA:1","ARSA:1","DGFI:1","Civil Service Regulations:1",
                      "BTZM:1","KNDF:1","KNPP:1","KNDO:1","DPLA:1","BGF:1","DCS:1","UWSA:1","PPNM:1","Russian Intelligence:1","CNA:1",
                      "CDF:1","CNF:1","United Nations:1","ULA:1","SOF:1","MDY:1","KBZ:1","SSPP:1","KTLA:1","NSIMS:1"]
}

# Flatten keyword patterns
compiled_patterns = []
for variants in KEYWORD_VARIANTS.values():
    for item in variants:
        parts = item.split(":")
        if len(parts) != 2:
            continue
        phrase, weight = parts[0].strip(), float(parts[1].strip())
        pat = re.compile(r"(?<!\w)"+re.escape(phrase)+r"(?!\w)", flags=re.IGNORECASE)
        compiled_patterns.append((phrase, weight, pat))

# Load JSON from extract_paragraphs.py
if not os.path.exists(json_file):
    raise FileNotFoundError(f"Input JSON not found: {json_file}")

with open(json_file, "r", encoding="utf-8") as f:
    paragraphs = json.load(f)

# Score paragraphs
scored = []
for entry in paragraphs:
    text = entry.get("text_snippet","").replace("**","")
    score_total = 0.0
    matched = {}
    for phrase, weight, pat in compiled_patterns:
        freq = len(pat.findall(text))
        if freq > 0:
            matched[phrase] = freq
            score_total += weight * freq
    entry_out = dict(entry)
    entry_out["score"] = round(score_total,6)
    entry_out["matched_keywords"] = matched
    entry_out["text_snippet"] = text
    scored.append(entry_out)

# Sort
scored.sort(key=lambda x:x["score"], reverse=True)

# Save cleaned JSON
out_json = os.path.join(OUTPUT_DIR, "paragraphs_scored_sorted_clean.json")
with open(out_json, "w", encoding="utf-8") as f:
    json.dump(scored, f, indent=2, ensure_ascii=False)

# Build Telegram-style HTML
html_out = [
    "<!doctype html>",
    "<html><head><meta charset='utf-8'>",
    "<title>Sorted Paragraphs</title>",
    "<style>body{font-family:Arial,sans-serif}.message{border:1px solid #ccc;margin:10px;padding:10px;border-radius:6px}.from_name{font-weight:bold;color:#2b4}.date{color:#666;font-size:0.85em}.text{margin:8px 0}.meta{font-size:0.8em;color:#888}.meta a{margin-left:10px;color:#06c;text-decoration:none}.meta a:hover{text-decoration:underline}</style>",
    "</head><body>"
]

for idx, e in enumerate(scored,1):
    if e["score"] <= 0:
        continue
    msg_id = e.get("message_id","").replace("message","")
    link_html = f'<a href="{ORIGINAL_FILE}#message{msg_id}" target="_blank">View Original</a>' if msg_id else ""
    html_out.append(f'<div class="message" id="msg{idx}">')
    html_out.append(f'  <div class="from_name">{e.get("username","N/A")}</div>')
    html_out.append(f'  <div class="date">{e.get("time","N/A")}</div>')
    html_out.append(f'  <div class="text">{e["text_snippet"]}</div>')
    html_out.append(f'  <div class="meta">Score: {e["score"]} | Words: {e.get("word_count","")} {link_html}</div>')
    html_out.append('</div>')

html_out.append("</body></html>")

out_html = os.path.join(OUTPUT_DIR, "final_paragraphs_sorted_like_telegram.html")
with open(out_html, "w", encoding="utf-8") as f:
    f.write("\n".join(html_out))

print("✅ Done.")
print("Scored JSON saved to:", out_json)
print("HTML saved to:", out_html)


[DEBUG] OUTPUT_DIR: /content/drive/MyDrive/html_keyword_output
[DEBUG] TARGET_FILE: messages_translated1.html
[DEBUG] ORIGINAL_FILE exists?: True
[DEBUG] JSON file path: /content/drive/MyDrive/html_keyword_output/final_paragraphs_sorted.json
[DEBUG] Summarized HTML file path: /content/drive/MyDrive/html_keyword_output/final_paragraphs_with_summary.html
✅ Done.
Scored JSON saved to: /content/drive/MyDrive/html_keyword_output/paragraphs_scored_sorted_clean.json
HTML saved to: /content/drive/MyDrive/html_keyword_output/final_paragraphs_sorted_like_telegram.html


In [ ]:
# === Summarized HTML Report (with t5-small + debug logs) ===
!pip -q install transformers sentencepiece

import os, json
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# --- Config ---
OUTPUT_DIR = "/content/drive/MyDrive/html_keyword_output"
ORIGINAL_FILE = "/content/drive/MyDrive/telegram_exports/messages_translated1.html"
original_file_url = "https://drive.google.com/file/d/1u9H-GJXpeWQH5J82vv56V4NuxFDe9mMy/view?usp=drive_link"
json_file = os.path.join(OUTPUT_DIR, "paragraphs_scored_sorted_clean.json")
summarized_html_file = os.path.join(OUTPUT_DIR, "final_paragraphs_with_summary.html")

print("[DEBUG] OUTPUT_DIR:", OUTPUT_DIR)
print("[DEBUG] JSON file:", json_file)
print("[DEBUG] ORIGINAL_FILE exists?:", os.path.exists(ORIGINAL_FILE))

# Load scored JSON
if not os.path.exists(json_file):
    raise FileNotFoundError(f"Expected JSON not found: {json_file}")

with open(json_file, "r", encoding="utf-8") as f:
    paragraphs = json.load(f)

print("[DEBUG] Loaded items:", len(paragraphs))

# Load t5-small summarizer
model_name = "t5-small"
print(f"[DEBUG] Loading summarizer model: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="pt", device=-1)  # CPU
print("[DEBUG] Summarizer ready (CPU, t5-small)")

def safe_summarize(text: str):
    """Summarize text using t5-small, keep short text unchanged."""
    if len(text.strip().split()) <= 50:
        return text.strip()
    try:
        prefixed_text = "summarize: " + text.strip()
        summary = summarizer(prefixed_text, max_length=80, min_length=20, do_sample=False)[0]["summary_text"]
        return summary
    except Exception as e:
        return f"[Summary failed: {e}]"

# Build summarized HTML
html_out = [
    "<!doctype html>",
    "<html><head><meta charset='utf-8'><title>Summarized Telegram Extracts</title>",
    "<style>",
    "body{font-family:Arial,sans-serif}",
    ".message{border:1px solid #ccc;padding:10px;margin:10px;border-radius:6px}",
    ".from_name{font-weight:bold;color:#2b4}",
    ".date{color:#666;font-size:0.85em}",
    ".text{margin:8px 0}",
    ".meta{font-size:0.85em;color:#666}",
    ".summary{background:#fff8d5;padding:8px;border-radius:6px;border:1px solid #eedc82}",
    "</style></head><body>"
]

summarized_count = 0
skipped_count = 0

for idx, entry in enumerate(paragraphs, 1):
    msg_id = entry.get("message_id","")
    text_snippet = entry.get("text_snippet","")
    word_count = entry.get("word_count", 0)
    needs_summary = word_count > 100

    # --- DEBUG ---
    preview = text_snippet[:50].replace("\n"," ") + ("..." if len(text_snippet) > 50 else "")
    print(f"[DEBUG] Paragraph {idx}: words={word_count}, needs_summary={needs_summary}, preview='{preview}'")

    html_out.append("<div class='message'>")
    html_out.append(f"  <div class='from_name'>{entry.get('username','N/A')}</div>")
    html_out.append(f"  <div class='date'>{entry.get('time','N/A')}</div>")
    html_out.append(f"  <div class='text'>{text_snippet}</div>")

    # Meta row with original paragraph score
    link_html = f"<a href='{original_file_url}#{msg_id}' target='_blank'>View Original</a>" if msg_id else ""
    html_out.append(f"  <div class='meta'>Score: {entry.get('score','')} | Words: {word_count} {('&nbsp;|&nbsp;' + link_html) if link_html else ''}</div>")

    # Summary block if needed
    if needs_summary:
        summarized_count += 1
        summary_text = safe_summarize(text_snippet)
        html_out.append(f"  <div class='summary'><b>Summary:</b> {summary_text}</div>")
    else:
        skipped_count += 1

    html_out.append("</div>")

html_out.append(f"<p style='color:#666;font-size:0.9em'>[INFO] Summaries created: {summarized_count} | Skipped (≤100 words): {skipped_count}</p>")
html_out.append("</body></html>")

# Save HTML
with open(summarized_html_file, "w", encoding="utf-8") as f:
    f.write("\n".join(html_out))

print("✅ Summarized HTML report saved to:", summarized_html_file)
print("[DONE]")


[DEBUG] OUTPUT_DIR: /content/drive/MyDrive/html_keyword_output
[DEBUG] JSON file: /content/drive/MyDrive/html_keyword_output/paragraphs_scored_sorted_clean.json
[DEBUG] ORIGINAL_FILE exists?: True
[DEBUG] Loaded items: 72
[DEBUG] Loading summarizer model: t5-small


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

[DEBUG] Paragraph 47: words=38, needs_summary=False, preview='  I watched the cartoon and the squirrels were agi...'
[DEBUG] Paragraph 48: words=18, needs_summary=False, preview=' I wouldn't 😂😂😂 have been happy if the head of the...'
[DEBUG] Paragraph 49: words=22, needs_summary=False, preview=' Burning KIA in Venmore.I'm  satisfied with what t...'
[DEBUG] Paragraph 50: words=8, needs_summary=False, preview='  NUG😂😂😂 Punches a Nutcracker and a Death Eater  '
[DEBUG] Paragraph 51: words=33, needs_summary=False, preview='  The Army Drone Squad is so sharp that it doesn't...'
[DEBUG] Paragraph 52: words=15, needs_summary=False, preview='  When the city of Demosso was captured,  the Army...'
[DEBUG] Paragraph 53: words=68, needs_summary=False, preview='  The convoy that  arrived at the gate yesterday m...'
[DEBUG] Paragraph 54: words=191, needs_summary=True, preview='  Body/Weapon Acquired (Strike) On the morning of ...'


Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Paragraph 55: words=28, needs_summary=False, preview=' Demosso  is occupied  by soldiers who beat the co...'
[DEBUG] Paragraph 56: words=60, needs_summary=False, preview='  The Army's mission is the defense of the State, ...'
[DEBUG] Paragraph 57: words=136, needs_summary=True, preview=' Violent PDFs have blocked the whole area...  This...'


Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Paragraph 58: words=271, needs_summary=True, preview=' Chairman of the Mandalay Solidarity and Developme...'
[DEBUG] Paragraph 59: words=13, needs_summary=False, preview=' The bridge was broken by the rebels of the Caucas...'
[DEBUG] Paragraph 60: words=64, needs_summary=False, preview='  If I wanted to be alone, I would drive out the p...'
[DEBUG] Paragraph 61: words=18, needs_summary=False, preview='  The girls under the military commission are only...'
[DEBUG] Paragraph 62: words=10, needs_summary=False, preview='  19-8-2025 Today, the Army completely recaptures ...'
[DEBUG] Paragraph 63: words=64, needs_summary=False, preview='  Do you have no self-confidence..?  Do you want t...'
[DEBUG] Paragraph 64: words=21, needs_summary=False, preview='  19-8-2025 After the total recapture of Dimosso b...'
[DEBUG] Paragraph 65: words=26, needs_summary=False, preview='  Awarding  of the first prize winner of the  Rain...'
[DEBUG] Paragraph 66: words=15, needs_summary=False, preview='  